# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 813ms/step - loss: 0.6710 - accuracy: 0.6168 - val_loss: 0.6603 - val_accuracy: 0.5512
Epoch 2/10
26/26 [==============================] - 20s 784ms/step - loss: 0.6051 - accuracy: 0.7384 - val_loss: 0.6245 - val_accuracy: 0.5951
Epoch 3/10
26/26 [==============================] - 21s 804ms/step - loss: 0.5353 - accuracy: 0.7689 - val_loss: 0.4984 - val_accuracy: 0.7756
Epoch 4/10
26/26 [==============================] - 21s 819ms/step - loss: 0.4733 - accuracy: 0.7932 - val_loss: 0.4384 - val_accuracy: 0.8146
Epoch 5/10
26/26 [==============================] - 22s 835ms/step - loss: 0.3944 - accuracy: 0.8455 - val_loss: 0.3849 - val_accuracy: 0.8146
Epoch 6/10
26/26 [==============================] - 21s 823ms/step - loss: 0.3404 - accuracy: 0.8759 - val_loss: 0.3370 - val_accuracy: 0.8439
Epoch 7/10
26/26 [==============================] - 22s 827ms/step - loss: 0.2772 - accuracy: 0.9051 - val_loss: 0.2385 - val_accuracy: 0.9220

In [11]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 1363), started 0:51:30 ago. (Use '!kill 1363' to kill it.)

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.69.h5
26/26 - 23s - loss: 0.6855 - accuracy: 0.5341 - val_loss: 0.6880 - val_accuracy: 0.4390
Epoch 2/5

Epoch 00002: saving model to weights.02-0.67.h5
26/26 - 22s - loss: 0.6703 - accuracy: 0.5645 - val_loss: 0.6742 - val_accuracy: 0.5073
Epoch 3/5

Epoch 00003: saving model to weights.03-0.65.h5
26/26 - 22s - loss: 0.6500 - accuracy: 0.6618 - val_loss: 0.6499 - val_accuracy: 0.6683
Epoch 4/5

Epoch 00004: saving model to weights.04-0.60.h5
26/26 - 22s - loss: 0.6191 - accuracy: 0.7105 - val_loss: 0.5999 - val_accuracy: 0.7512
Epoch 5/5

Epoch 00005: saving model to weights.05-0.55.h5
26/26 - 22s - loss: 0.5702 - accuracy: 0.7713 - val_loss: 0.5476 - val_accuracy: 0.7268


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 27s - loss: 0.6768 - accuracy: 0.5560 - val_loss: 0.6863 - val_accuracy: 0.4439


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 22s - loss: 0.6780 - accuracy: 0.5669 - val_loss: 0.6630 - val_accuracy: 0.6195
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 22s - loss: 0.6482 - accuracy: 0.6496 - val_loss: 0.6320 - val_accuracy: 0.7805


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 22s - loss: 0.6701 - accuracy: 0.5949 - val_loss: 0.6637 - val_accuracy: 0.5171
Epoch 2/50
26/26 - 20s - loss: 0.6281 - accuracy: 0.7129 - val_loss: 0.6184 - val_accuracy: 0.6585
Epoch 3/50
26/26 - 20s - loss: 0.5787 - accuracy: 0.7287 - val_loss: 0.5878 - val_accuracy: 0.6488
Epoch 4/50
26/26 - 20s - loss: 0.5091 - accuracy: 0.7822 - val_loss: 0.4796 - val_accuracy: 0.8341
Epoch 5/50
26/26 - 20s - loss: 0.4256 - accuracy: 0.8394 - val_loss: 0.3659 - val_accuracy: 0.8878
Epoch 6/50
26/26 - 20s - loss: 0.3712 - accuracy: 0.8735 - val_loss: 0.3150 - val_accuracy: 0.9024
Epoch 7/50
26/26 - 20s - loss: 0.3102 - accuracy: 0.8844 - val_loss: 0.3391 - val_accuracy: 0.8732
Epoch 8/50
26/26 - 20s - loss: 0.2468 - accuracy: 0.9185 - val_loss: 0.1928 - val_accuracy: 0.9659
Epoch 9/50
26/26 - 19s - loss: 0.1977 - accuracy: 0.9355 - val_loss: 0.1828 - val_accuracy: 0.9512
Epoch 10/50
26/26 - 20s - loss: 0.1696 - accuracy: 0.9513 - val_loss: 0.1721 - val_accuracy: 0.9463
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 799ms/step - loss: 0.6573 - accuracy: 0.5803 - val_loss: 0.6114 - val_accuracy: 0.7512
Epoch 2/5
26/26 [==============================] - 20s 775ms/step - loss: 0.5890 - accuracy: 0.7019 - val_loss: 0.5483 - val_accuracy: 0.7610
Epoch 3/5
26/26 [==============================] - 20s 769ms/step - loss: 0.5400 - accuracy: 0.7202 - val_loss: 0.6074 - val_accuracy: 0.6244
Epoch 4/5
26/26 [==============================] - 20s 765ms/step - loss: 0.4917 - accuracy: 0.7616 - val_loss: 0.4572 - val_accuracy: 0.7951
Epoch 5/5
26/26 [==============================] - 20s 759ms/step - loss: 0.4109 - accuracy: 0.8382 - val_loss: 0.4207 - val_accuracy: 0.8244


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.580292,0.658058,0.751219,0.611396
1,1,0.701947,0.589347,0.760976,0.548254
2,2,0.720195,0.541142,0.624390,0.607408
3,3,0.761557,0.491399,0.795122,0.457193
4,4,0.838200,0.410809,0.824390,0.420689


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 30s 1s/step - loss: 0.6745 - accuracy: 0.5693 - val_loss: 0.6668 - val_accuracy: 0.5854

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 20s 772ms/step - loss: 0.6483 - accuracy: 0.6363 - val_loss: 0.6440 - val_accuracy: 0.7463

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 765ms/step - loss: 0.6329 - accuracy: 0.7019 - val_loss: 0.6484 - val_accuracy: 0.6098

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 781ms/step - loss: 0.6268 - accuracy: 0.7117 - val_loss: 0.6425 - val_accuracy: 0.6293

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 776ms/step - loss: 0.6228

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 828ms/step - loss: 0.6766 - accuracy: 0.5657 - val_loss: 0.7020 - val_accuracy: 0.4341
Epoch 2/50
26/26 [==============================] - 20s 777ms/step - loss: 0.6404 - accuracy: 0.6509 - val_loss: 0.6467 - val_accuracy: 0.5659
Epoch 3/50
26/26 [==============================] - 20s 769ms/step - loss: 0.5892 - accuracy: 0.7214 - val_loss: 0.5592 - val_accuracy: 0.7561
Epoch 4/50
25/26 [===========================>..] - ETA: 0s - loss: 0.5245 - accuracy: 0.7600
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 20s 765ms/step - loss: 0.5220 - accuracy: 0.7616 - val_loss: 0.6475 - val_accuracy: 0.5805
Epoch 5/50
26/26 [==============================] - 20s 777ms/step - loss: 0.4735 - accuracy: 0.7908 - val_loss: 0.4839 - val_accuracy: 0.8049
Epoch 6/50
26/26 [==============================] - 20s 769ms/step - loss: 0.4532 - accuracy: 0.8054 - val_loss: 0.4622 - 

Epoch 46/50
26/26 [==============================] - 20s 763ms/step - loss: 0.1807 - accuracy: 0.9623 - val_loss: 0.1835 - val_accuracy: 0.9463
Epoch 47/50
25/26 [===========================>..] - ETA: 0s - loss: 0.1758 - accuracy: 0.9638
Epoch 00047: ReduceLROnPlateau reducing learning rate to 0.001.
26/26 [==============================] - 20s 768ms/step - loss: 0.1758 - accuracy: 0.9647 - val_loss: 0.1881 - val_accuracy: 0.9366
Epoch 48/50
25/26 [===========================>..] - ETA: 0s - loss: 0.1728 - accuracy: 0.9675
Epoch 00048: ReduceLROnPlateau reducing learning rate to 0.001.
26/26 [==============================] - 20s 781ms/step - loss: 0.1725 - accuracy: 0.9684 - val_loss: 0.1878 - val_accuracy: 0.9366
Epoch 49/50
26/26 [==============================] - 20s 773ms/step - loss: 0.1692 - accuracy: 0.9659 - val_loss: 0.1713 - val_accuracy: 0.9512
Epoch 50/50
26/26 [==============================] - 20s 785ms/step - loss: 0.1635 - accuracy: 0.9672 - val_loss: 0.1651 - val_acc

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6006 (pid 8436), started 0:16:48 ago. (Use '!kill 8436' to kill it.)